In [1]:
import pandas as pd
df = pd.read_csv('scoring_3_1_edus.csv')

In [45]:
#pd.read_csv('scoring_3_1_edus.csv')[]

#df.loc[[0, 1, 2, 3, 200, 201]]
#df['Model'].unique()
r_df = df[df['Model'] == 'RoBERTa FT']
r_df[r_df['Method'] == 'K Center']

,Unnamed: 0.1,Unnamed: 0,Method,Summary,Label,rouge1,rouge2,rougeL,rougeLsum,Model
88,8,8,K Center,['5ft (4.4m) high bridge.30am on March 13 when...,Transporter driver had taken a short-cut to av...,0.285714,0.058252,0.171429,0.171429,RoBERTa FT
92,12,12,K Center,"["" But Parish believes there is no need for th...",Crystal Palace co-chairman says club are in no...,0.317073,0.050000,0.170732,0.170732,RoBERTa FT
96,16,16,K Center,"[""A federal judge shot in the leg outside his...",Federal judge Terrence Berg was shot in the le...,0.377358,0.230769,0.339623,0.339623,RoBERTa FT
100,32,32,K Center,['’ Thefts on the RER train system are not jus...,Chinese embassy in Paris took the rare step of...,0.222222,0.075949,0.148148,0.148148,RoBERTa FT
104,36,36,K Center,"["" Scroll down for video . Toby first made wav...","Toby Huntington-Whiteley, 24, stars in televis...",0.212766,0.000000,0.127660,0.127660,RoBERTa FT
108,44,44,K Center,"[""Muhammad Ali wants Manny Pacquiao to emerge ...",Muhammad Ali's daughter says her father admire...,0.383838,0.082474,0.262626,0.262626,RoBERTa FT
112,72,72,K Center,[' Asked to choose between the Tories and Labo...,"If economy stays on track until the Election, ...",0.436782,0.235294,0.344828,0.344828,RoBERTa FT
116,76,76,K Center,[' Scroll down for video . Cholita (pictured) ...,Cholita lost most of her hair because of the s...,0.320755,0.134615,0.245283,0.245283,RoBERTa FT
120,80,80,K Center,"["" Blackburn defender Matthew Kilgallon (left)...",Liverpool played out a 0-0 draw with Blackburn...,0.246154,0.015625,0.123077,0.123077,RoBERTa FT
124,92,92,K Center,"["" Scroll down for video .' A number of users ...",32-page Taste of Israel insert was included in...,0.136986,0.028169,0.109589,0.109589,RoBERTa FT


In [55]:

df_dict = {
}

df['Label'] = df['Label'].str.replace('\n', ' ')

for label in df['Label'].unique():
    cur_df = df[df['Label']==label].reset_index()
    i = int(cur_df['index'][0] / 4)
    for j in range(0, 3):
        roberta = cur_df.loc[j]
        roberta_ft = cur_df.loc[j+4]
        roberta_ft_edus = cur_df.loc[j+8]
        metric='rouge1'
        if roberta[metric] <= roberta_ft[metric] and roberta_ft[metric] <= roberta_ft_edus[metric]:
            method = roberta['Method']
            if method == 'Random':
                continue
            if not df_dict.get(str(i)):
                df_dict[str(i)] = {}
            df_dict[str(i)][method] = {}
            df_dict[str(i)][method]['roberta_score'] = roberta[metric]
            df_dict[str(i)][method]['roberta_summary'] = roberta['Summary']
            df_dict[str(i)][method]['roberta_ft_score'] = roberta_ft[metric]
            df_dict[str(i)][method]['roberta_ft_summary'] = roberta_ft['Summary']
            df_dict[str(i)][method]['roberta_ft_edus_score'] = roberta_ft_edus[metric]
            df_dict[str(i)][method]['roberta_ft_edus_summary'] = roberta_ft_edus['Summary']
            df_dict[str(i)][method]['real_summary'] = roberta['Label']



In [56]:
import json
with open('results_3.json', 'w') as f:
    f.write(json.dumps(df_dict, indent=4))

In [2]:
import datasets

/opt/homebrew/Caskroom/miniconda/base/envs/tml_base/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
ds = datasets.load_from_disk('data_parsed_berta')

In [26]:
len(ds)

194

In [27]:
df = ds.to_pandas()
df['article'] = df.groupby(['article_id', 'highlight']).transform(lambda x: ','.join(x))
df.drop_duplicates(subset=['article'], inplace=True)
#df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)
df[['article_id', 'article', 'highlight']].to_csv('input/data_parsed_roberta_joined.csv', index=False)

In [17]:
import pandas as pd
from ast import literal_eval
df = pd.read_csv('../NeuralEDUSeg/data/results/data_parsed_roberta_joined.csv', converters={'edus': literal_eval})
    
df_dict = {'article_id': [], 'article': [], 'highlight': [], 'sentence': []}
for index, row in df.iterrows():
    sentences = row['edus']
    for sentence in sentences:
        df_dict['article_id'].append(row['article_id'])
        df_dict['article'].append(row['article'])
        df_dict['highlight'].append(row['highlight'])
        df_dict['sentence'].append(sentence)

In [18]:
new_df = pd.DataFrame.from_dict(df_dict)
new_df


,article_id,article,highlight,sentence
0,8d9d1810ac45ee826a8d1f1d473596c0667d7c7f,DNA molecules extracted from fish sperm is bei...,Researchers at the Polytechnic University of T...,DNA molecules
1,8d9d1810ac45ee826a8d1f1d473596c0667d7c7f,DNA molecules extracted from fish sperm is bei...,Researchers at the Polytechnic University of T...,extracted from fish sperm
2,8d9d1810ac45ee826a8d1f1d473596c0667d7c7f,DNA molecules extracted from fish sperm is bei...,Researchers at the Polytechnic University of T...,is being used
3,8d9d1810ac45ee826a8d1f1d473596c0667d7c7f,DNA molecules extracted from fish sperm is bei...,Researchers at the Polytechnic University of T...,to develop new types of flame retardant materi...
4,8d9d1810ac45ee826a8d1f1d473596c0667d7c7f,DNA molecules extracted from fish sperm is bei...,Researchers at the Polytechnic University of T...,Scientists recently discovered
...,...,...,...,...
4881,1eb6ee56e4bfed04001614c2988a09f1e49d7da2,Swansea manager Garry Monk believes the only w...,Swansea manager Garry Monk believes Tim Sherwo...,because he 's got the talent
4882,1eb6ee56e4bfed04001614c2988a09f1e49d7da2,Swansea manager Garry Monk believes the only w...,Swansea manager Garry Monk believes Tim Sherwo...,"to do that ,"
4883,1eb6ee56e4bfed04001614c2988a09f1e49d7da2,Swansea manager Garry Monk believes the only w...,Swansea manager Garry Monk believes Tim Sherwo...,' Sinclair has thrived
4884,1eb6ee56e4bfed04001614c2988a09f1e49d7da2,Swansea manager Garry Monk believes the only w...,Swansea manager Garry Monk believes Tim Sherwo...,since arriving at Aston Villa


In [19]:

ds = datasets.Dataset.from_pandas(new_df)
print(ds)
ds.save_to_disk('data_parsed_roberta_edus')

Dataset({
    features: ['article_id', 'article', 'highlight', 'sentence'],
    num_rows: 4886
})


Saving the dataset (1/1 shards): 100%|██████████| 4886/4886 [00:00<00:00, 255703.65 examples/s]
